In [33]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def pick(match_id):
    pd.set_option('display.max_columns', None)
    # 读取数据并量化
    df = pd.read_csv ('/Users/lycoris/Desktop/other/github/MCM2024/Data/Wimbledon_featured_matches.csv')
    df = df[df['match_id'] == match_id].copy()
    for i, score in enumerate(['15', '30', '40', 'AD']):
        df.loc[(df.p1_score== score) , 'p1_score'] = i + 1
        df.loc[(df.p2_score== score) , 'p2_score'] = i + 1
    df.ffill(inplace=True)
    df[ 'p1_score' ] = df['p1_score' ].astype(int)
    df[ 'p2_score' ] = df['p2_score'].astype(int)    

    # 提取所需特证
    x1_ls,x2_ls,x3_ls,x4_ls,x5_ls,x6_ls,x7_ls,x8_ls,x9_ls,x10_ls,\
    x12_ls,x13_ls,x14_ls,x15_ls,x16_ls=[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
    label_ls = []
    label = 0

    for match_id,set_no,game_no,point_no in zip(df.match_id, df.set_no, df.game_no, df.point_no):
            match = df[df.match_id == match_id]
            set_ = match[ match.set_no == set_no]
            game_ = set_[set_.game_no == game_no]
            point_ = game_[game_.point_no == point_no]

            # 如果x>0，则对p1有利，如果x<0，则对p2有利
            x1 = point_[ 'p1_games' ].values[0]-point_[ 'p2_games' ].values[0] # p1局领先进度
            x2 = point_[ 'p1_score' ].values[0]- point_[ 'p2_score' ].values[0] # p1小分领先进度
            x3 = 1 if point_[ 'serve_no' ].values[0] == 1 else 0 # p1是否发球
            x4 = 1 if label==1 else 0 # p1上一小分是否得到
            x5 = point_[ 'p1_sets' ].values[0] - point_[ 'p2_sets' ].values[0] # p1盘领先进度
            # p1连胜局数
            x6 = 1 if 1 in game_['p1_ace'].values else 0
            x7 = 1 if 1 in game_['p1_winner'].values else 0
            x8 = 1 if 1 in game_['p1_double_fault'].values else 0
            x9 = 1 if 1 in game_['p1_unf_err'].values else 0
            x10 = game_[ 'p1_net_pt_won' ]. sum()/game_[ 'p1_net_pt' ].sum() if game_[ 'p1_net_pt' ].sum() != 0 else 0

            index = match.index.tolist( ).index(point_.index.tolist()[0])
            x12 = match.iloc [ : index+1][ 'p1_distance_run' ].sum()
            x13 = match.iloc [ index-2 :index+1][ 'p1_distance_run' ].sum()
            x14 = point_[ 'p1_distance_run' ].values[0]
            x15 = point_[ 'speed_mph' ].values[0]
            x16 = x15 * x3

            label = 1 if point_[ 'point_victor' ].values[0] == 1 else 0
            label_ls.append(label)
            x1_ls.append(x1)
            x2_ls.append(x2)
            x3_ls.append(x3)
            x4_ls.append(x4)
            x5_ls.append(x5)
            x6_ls.append(x6)
            x7_ls.append(x7)
            x8_ls.append(x8)
            x9_ls.append(x9)
            x10_ls.append(x10)
            x12_ls.append(x12)
            x13_ls.append(x13)
            x14_ls.append(x14)
            x15_ls.append(x15)
            x16_ls.append(x16)



    dataset = pd.DataFrame({'x1': x1_ls, 'x2': x2_ls, 'x3': x3_ls, 'x4': x4_ls, 'x5': x5_ls, 'x6': x6_ls, 'x7': x7_ls, 'x8': x8_ls, 'x9': x9_ls, 'x10': x10_ls, 'x12': x12_ls, 'x13': x13_ls, 'x14': x14_ls, 'x15': x15_ls, 'x16': x16_ls,
                            'label': label_ls})
    # dataset.to_csv('/Users/lycoris/Desktop/other/github/MCM2024/Data/Dataset.csv')

    # 归一化数据
    scaler = MinMaxScaler()
    cols = dataset.columns[:-1]
    scaler.fit(dataset[cols].values)
    dataset[cols] = scaler.transform(dataset[cols].values)
    # dataset.to_csv('/Users/lycoris/Desktop/other/github/MCM2024/Data/ScaledDataset.csv')

    return dataset

In [34]:
tryset = pick("2023-wimbledon-1301")
tryset.to_csv('/Users/lycoris/Desktop/other/github/MCM2024/Data/tryset.csv')